In [17]:
from autoemulate.experimental.sensitivity_analysis import SensitivityAnalysis
from autoemulate.experimental.compare import AutoEmulate
from autoemulate.experimental.emulators import *
from autoemulate.experimental.simulations.flow_problem import FlowProblem
from autoemulate.experimental.simulations.projectile import ProjectileMultioutput

In [2]:
param_ranges = {"T": (0.5, 2.0),
                "td": (0.1, 0.5),
                "amp": (100.0, 1000.0),
                "dt": (0.0001, 0.01),
                "ncycles": (5, 20),
                "ncomp": (5, 20),
                "C": (20.0, 60.0),
                "R": (0.01, 0.1),
                "L": (0.001, 0.005),
                "R_o": (0.01, 0.05),
                "p_o": (5.0, 15.0)}
output_names = ["pressure"]

simulator = FlowProblem(
    param_ranges=param_ranges,
    output_names=output_names,
)

In [3]:
input_samples = simulator.sample_inputs(10)
input_samples.shape

torch.Size([10, 11])

In [4]:
input_samples

tensor([[1.3781e+00, 3.7993e-01, 9.5719e+02, 3.7398e-03, 1.3305e+01, 1.4198e+01,
         5.3651e+01, 1.4842e-02, 3.7284e-03, 2.9434e-02, 9.1827e+00],
        [1.4582e+00, 1.6033e-01, 3.7194e+02, 1.0135e-03, 5.1452e+00, 7.1387e+00,
         2.4778e+01, 3.4315e-02, 1.4927e-03, 4.4667e-02, 1.3904e+01],
        [1.6167e+00, 4.1322e-01, 6.6905e+02, 6.8606e-03, 6.5592e+00, 1.0466e+01,
         2.0907e+01, 7.2993e-02, 2.6707e-03, 3.7285e-02, 7.7396e+00],
        [7.2377e-01, 2.7410e-01, 5.6092e+02, 8.4794e-03, 1.0835e+01, 1.5548e+01,
         5.8571e+01, 7.8491e-02, 1.8622e-03, 1.2684e-02, 1.2807e+01],
        [9.1336e-01, 3.0583e-01, 1.7642e+02, 9.8507e-03, 1.5882e+01, 1.3295e+01,
         3.4486e+01, 5.9645e-02, 4.8418e-03, 4.0434e-02, 1.4075e+01],
        [5.9208e-01, 4.4897e-01, 3.6939e+02, 5.8084e-03, 1.8631e+01, 1.7417e+01,
         4.8142e+01, 3.9636e-02, 1.1513e-03, 1.6886e-02, 1.1305e+01],
        [1.8823e+00, 1.8618e-01, 8.7706e+02, 7.7568e-03, 1.7760e+01, 1.1019e+01,
         3.93

In [5]:
single_output = simulator.forward(
    input_samples[0:1]
)

print(single_output)
single_output.shape

tensor([[ 13.0053,   2.5250,  12.9668,  10.0925,  12.9376,  22.8323,  12.9589,
          38.8043,  13.0251,  58.4594,  13.1039,  80.8184,  13.1487, 101.3486,
          13.1405, 114.4052,  13.0616, 117.3930,  12.9146, 115.4014,  12.7216,
         110.0404,  12.5281, 104.0501,  12.3685, 101.2799,  12.2232,  99.6762]])


torch.Size([1, 28])

In [26]:
print(simulator.in_dim)
print(simulator.param_names)
print(simulator.param_bounds)
print(simulator.output_names)

11
['T', 'td', 'amp', 'dt', 'ncycles', 'ncomp', 'C', 'R', 'L', 'R_o', 'p_o']
[(0.5, 2.0), (0.1, 0.5), (100.0, 1000.0), (0.0001, 0.01), (5, 20), (5, 20), (20.0, 60.0), (0.01, 0.1), (0.001, 0.005), (0.01, 0.05), (5.0, 15.0)]
['pressure']


In [29]:
simulator = ProjectileMultioutput()
x = simulator.sample_inputs(100)
y = simulator.forward_batch(x)

Running simulations: 100%|██████████| 100/100 [00:00<00:00, 1.14ksample/s]


In [30]:
x.shape, y.shape

(torch.Size([100, 2]), torch.Size([100, 2]))

In [31]:
ae = AutoEmulate(x, y, models=[RandomForest])

Comparing models: 100%|██████████| 1.00/1.00 [00:05<00:00, 5.60s/model]


In [32]:
best = ae.best_result()
model = best.model

In [33]:
problem = {
    'num_vars': simulator.in_dim,
    'names': simulator.param_names,
    'bounds': simulator.param_bounds,
    'output_names': simulator.output_names,
    }
sa = SensitivityAnalysis(model, problem=problem)

In [34]:
df = sa.run("sobol")